In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.agents.a3c.a3c_tf_policy import A3CTFPolicy
from ray.rllib.agents.a3c import a3c
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d import MyEnv
#from test_env_for_lstm import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions
#from modules.models import MyConv2DModel_v0B_Small_CBAM_1DConv_Share
#from modules.models import MyRNNUAVClass
#from modules.models import DenseNetModelLarge
from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
from utility.save_logs import save_logs
from utility.save_logs import save_hists
from utility.save_logs import save_env_info

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 1
CONTINUAL = False
NUM_EVAL = 1
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

#ModelCatalog.register_custom_model('my_model', MyRNNUAVClass)

eval_env = MyEnv()
policies = {
    #"blue_1": PolicySpec(config={"gamma": 0.99}),
    #"blue_2": PolicySpec(config={"gamma": 0.95}),
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": True,"use_lstm": True,"max_seq_len": 200},}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": True,"use_lstm": True,"max_seq_len": 200},}),
    #"blue_0": (A3CTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    #"blue_1": (A3CTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
}
policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env
config = ppo.DEFAULT_CONFIG.copy()
config = {"env": MyEnv,"num_gpus": 0,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 0,
          "train_batch_size": 600*5,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          #"clip_actions":True,"normalize_actions":True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "explore":True,
          "sgd_minibatch_size": 300, "num_sgd_iter":20,
          "exploration_config": {"type": "StochasticSampling","random_timesteps":0}, #PPO デフォルト "random_timesteps":0
          #"model":{"fcnet_activation": "relu","fcnet_hiddens": [256, 256, 256],"post_fcnet_activation": "linear",
          #         "vf_share_layers": True,},#"linear","relu","tanh" "use_lstm":True,"lstm_cell_size":256,"max_seq_len":128
          
          #"model": {"custom_model": "my_model"},
          "multiagent": {"policies": policies,  "policy_mapping_fn": policy_mapping_fn}
         }
#res_name = "sgd"+str(config["sgd_minibatch_size"])+"sgd_num"+str(config["num_sgd_iter"])+"lr"+str(config["lr"])+"gamma"+str(config["gamma"])
res_name = "test"
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    trainer.restore(checkpoint_path=model_path)

models_dir = os.path.join('./' + PROJECT + '/models/')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
for j in range(2):
    text_name = models_dir + TRIAL + "blue_"+str(j) +'.txt'
    with open(text_name, "w") as fp:
        trainer.get_policy("blue_"+str(j)).model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
    png_name = models_dir + TRIAL + '.png'
    plot_model(trainer.get_policy("blue_"+str(j)).model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):
C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
2022-03-16 22:32:36,485	INFO trainer.py:723 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also want to then set `eager_tracing=True` in order to reach similar execution speed as with static-graph mode.
2022-03-16 22:32:36,486	INFO ppo.py:167 -- In multi-agent mode, policies will be optimized sequentially by the mult

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar  7 21:50:01 2022

@author: Takumi
"""
eval_env.reset()
save_env_info(eval_env)
record_mode = 0
results_dir = os.path.join('./' + PROJECT + '/results/')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.eval = False
    eval_env.reset()
    results = trainer.train()
    save_logs(res_name,results,steps,CONTINUAL)
    print(pretty_print(results))
    #check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n-------------- Evaluation at steps:{steps} starting ! --------------')
        #print(pretty_print(results))
        check_point = trainer.save(checkpoint_dir=check_point_dir)
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
            trainer.restore(checkpoint_path=model_path)
            eval_env.eval = True
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            #fig = plt.figure(1,figsize=(8.0, 6.0))
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100

            cell_size = 256
            state_0=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            state_1=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            action_dict0 = [0,0]
            action_dict1 = [0,0]
            rewards = {"blue_0":0,"blue_1":0}
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                #video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

            while True:
                action_dict = {}
                #for j in range(eval_env.blue_num):
                    #if not eval_env.blue[j].hitpoint == 0:
                    #action_dict['blue_' + str(j)] = trainer.compute_action(obs['blue_' + str(j)])
                    #action_dict['blue_' + str(j)] = trainer.compute_single_action(obs['blue_' + str(j)],policy_id='blue_' + str(j),
                    #                                                   clip_actions=True,explore=False)
                    #action_dict['blue_' + str(j)] = trainer.compute_single_action(obs['blue_' + str(j)],policy_id='blue_' + str(j))

                action_dict0 = trainer.compute_single_action(obs['blue_0'],
                                                             state=state_0,prev_action=None,prev_reward=None,
                                                             policy_id='blue_0',explore=False)
                action_dict1 = trainer.compute_single_action(obs['blue_1'],
                                                             state=state_1,prev_action=None,prev_reward=None,
                                                             policy_id='blue_1',explore=False)
                
                #action_dict0 = trainer.compute_single_action(obs['blue_0'],policy_id='blue_0')
                #action_dict1 = trainer.compute_single_action(obs['blue_1'],policy_id='blue_1')
                state_0 = action_dict0[1]
                state_1 = action_dict1[1]
                obs, rewards, dones, infos = eval_env.step({'blue_0': action_dict0[0], 'blue_1': action_dict1[0]})

                env_blue_pos_temp_mod, env_red_pos_temp_mod, env_mrm_pos_temp_mod = render_env.copy_from_env_mod(eval_env)
                if eval_env.timer == 1:
                    env_blue_pos_mod = env_blue_pos_temp_mod
                    env_red_pos_mod = env_red_pos_temp_mod
                    env_mrm_pos_mod = env_mrm_pos_temp_mod
                else:
                    env_blue_pos_mod = np.vstack([env_blue_pos_mod,env_blue_pos_temp_mod])
                    env_red_pos_mod = np.vstack([env_red_pos_mod,env_red_pos_temp_mod])
                    env_mrm_pos_mod = np.vstack([env_mrm_pos_mod,env_mrm_pos_temp_mod])

                # plt.clf()

                # plt.subplots_adjust(left=-0.1,right=1.1,bottom=-0.1,top=1.1)
                # fig.canvas.draw()
                # plt.pause(.01)

                #if record_mode == 0:
                    #img = np.array(fig.canvas.renderer.buffer_rgba())
                    #img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    #video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()
                    
                #env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp = render_env.copy_from_env(eval_env)
                
                #env_blue_pos.append(env_blue_pos_temp)
                #env_red_pos.append(env_red_pos_temp)
                #env_mrm_pos.append(env_mrm_pos_temp)
                #step_num = step_num + 1
                # エピソードの終了処理
                if dones['__all__']:
                    save_hists("blue",steps,env_blue_pos_mod)
                    save_hists("red",steps,env_red_pos_mod)
                    save_hists("mrm",steps,env_mrm_pos_mod)
                    # print(f'all done at {env.steps}')
                    break
                
            #del env_blue_pos[0]
            #del env_red_pos[0]
            #del env_mrm_pos[0]
            
            #hist_blue_pos = np.vstack(env_blue_pos)
            #hist_red_pos = np.vstack(env_red_pos)
            #hist_mrm_pos = np.vstack(env_mrm_pos)
            
            #f = open(results_file,'wb')
            #pickle.dump(emv_blue_pos,f)
            #pickle.dump(emv_red_pos,f)
            #pickle.dump(emv_mrm_pos,f)
            #f.close()
            
            #if record_mode == 0:
               # video.release()

ray.shutdown()

-------------------------- Scene: 1 --------------------------

----------------- Training at steps:0 start! -----------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 1 --------------------------
TIME LIMIT LOSE
blue_0 False False 744 -0.05999999999999999 -5.549999999999977
blue_1 False False 744 -0.2296 -2.832600000000008
-------------------------- Scene: 0 --------------------------
TIME LIMIT LOSE
blue_0 False False 692 -0.17 15.799999999999871
blue_1 False False 692 0.02 4.157600000000026
-------------------------- Scene: 0 --------------------------
TIME LIMIT LOSE
blue_0 False False 567 -0.2 13.439999999999985
blue_1 False False 567 0.0 4.668600000000014
-------------------------- Scene: 0 --------------------------
TIME LIMIT LOSE
blue_0 False False 617 -0.04 12.820000000000043
blue_1 False False 617 -0.2296 5.756599999999977
-------------------------- Scene: 0 --------------------------
44 blue_1 DOWN
TIME LIMIT LOSE
blue

2022-03-16 22:37:17,264	WARNING deprecation.py:46 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!


agent_timesteps_total: 6278
custom_metrics: {}
date: 2022-03-16_22-38-40
done: false
episode_len_mean: 627.8
episode_media: {}
episode_reward_max: 19.957599999999946
episode_reward_mean: 13.006039999999956
episode_reward_min: -8.38259999999997
episodes_this_iter: 5
episodes_total: 5
experiment_id: bdd7dc4555b24fc0a05a244306915927
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 4.663570404052734
        entropy_coeff: 0.0
        kl: 0.06019717827439308
        model: {}
        policy_loss: -0.004756579175591469
        total_loss: 5.764533519744873
        vf_explained_var: -0.33776816725730896
        vf_loss: 5.757250785827637
    blue_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 3.8982558250427246
        entropy_coeff: 0.0
        kl: 0.06379

2022-03-16 22:38:40,943	INFO trainable.py:468 -- Restored on 127.0.0.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_000001\checkpoint-1
2022-03-16 22:38:40,944	INFO trainable.py:475 -- Current state after restoring: {'_iteration': 1, '_timesteps_total': 0, '_time_total': 352.4931299686432, '_episodes_total': 5}


-------------------------- Scene: 0 --------------------------
TIME LIMIT LOSE
blue_0 False False 373 -0.04999999999999999 -4.059999999999997
blue_1 False False 373 -0.21960000000000002 -1.8168000000000006

----------------- Training at steps:1 start! -----------------
-------------------------- Scene: 1 --------------------------
TIME LIMIT LOSE
blue_0 False False 726 -0.23 21.050000000000047
blue_1 False False 726 0.0 7.129999999999937
-------------------------- Scene: 0 --------------------------
TIME LIMIT LOSE
blue_0 False False 679 -0.23 9.639999999999954
blue_1 False False 679 -0.01 0.3306000000000102
-------------------------- Scene: 0 --------------------------
TIME LIMIT LOSE
blue_0 False False 723 -0.23 18.440000000000037
blue_1 False False 723 0.0 5.389999999999973
-------------------------- Scene: 0 --------------------------
TIME LIMIT LOSE
blue_0 False False 553 0.01 0.549999999999996
blue_1 False False 553 -0.17960000000000004 4.198399999999984
-------------------------

...

agent_timesteps_total: 175940
custom_metrics: {}
date: 2022-03-17_01-29-44
done: false
episode_len_mean: 581.75
episode_media: {}
episode_reward_max: 40.10000000000028
episode_reward_mean: 17.613093999999954
episode_reward_min: -36.07920000000001
episodes_this_iter: 6
episodes_total: 147
experiment_id: bdd7dc4555b24fc0a05a244306915927
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 29.19292640686035
        cur_lr: 0.0002500000118743628
        entropy: 5.497603893280029
        entropy_coeff: 0.0
        kl: 0.009127476252615452
        model: {}
        policy_loss: -0.023004136979579926
        total_loss: 1.4499810934066772
        vf_explained_var: 0.04158946871757507
        vf_loss: 1.2065273523330688
    blue_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 4.324877738952637
        cur_lr: 0.0002500000118743628
        entropy: 4.294979572296143
        entropy_coeff: 0.0
        kl: 0.007

2022-03-17 01:36:58,725	INFO trainable.py:468 -- Restored on 127.0.0.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_000027\checkpoint-27
2022-03-17 01:36:58,726	INFO trainable.py:475 -- Current state after restoring: {'_iteration': 27, '_timesteps_total': 0, '_time_total': 10011.395394563675, '_episodes_total': 153}


agent_timesteps_total: 182670
custom_metrics: {}
date: 2022-03-17_01-36-58
done: false
episode_len_mean: 580.81
episode_media: {}
episode_reward_max: 40.10000000000028
episode_reward_mean: 17.278037999999952
episode_reward_min: -36.07920000000001
episodes_this_iter: 6
episodes_total: 153
experiment_id: bdd7dc4555b24fc0a05a244306915927
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 29.19292640686035
        cur_lr: 0.0002500000118743628
        entropy: 5.559453964233398
        entropy_coeff: 0.0
        kl: 0.013286776840686798
        model: {}
        policy_loss: -0.02054213546216488
        total_loss: 1.8850429058074951
        vf_explained_var: 0.18129903078079224
        vf_loss: 1.5177050828933716
    blue_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 4.324877738952637
        cur_lr: 0.0002500000118743628
        entropy: 4.2642951011657715
        entropy_coeff: 0.0
        kl: 0.012

2022-03-17 01:44:01,076	INFO trainable.py:468 -- Restored on 127.0.0.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_000028\checkpoint-28
2022-03-17 01:44:01,076	INFO trainable.py:475 -- Current state after restoring: {'_iteration': 28, '_timesteps_total': 0, '_time_total': 10399.724871873856, '_episodes_total': 158}


agent_timesteps_total: 189174
custom_metrics: {}
date: 2022-03-17_01-44-00
done: false
episode_len_mean: 586.04
episode_media: {}
episode_reward_max: 40.10000000000028
episode_reward_mean: 17.186037999999954
episode_reward_min: -36.07920000000001
episodes_this_iter: 5
episodes_total: 158
experiment_id: bdd7dc4555b24fc0a05a244306915927
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 29.19292640686035
        cur_lr: 0.0002500000118743628
        entropy: 5.487540245056152
        entropy_coeff: 0.0
        kl: 0.008658671751618385
        model: {}
        policy_loss: -0.06862473487854004
        total_loss: 1.4391837120056152
        vf_explained_var: 0.002125880680978298
        vf_loss: 1.2550365924835205
    blue_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 4.324877738952637
        cur_lr: 0.0002500000118743628
        entropy: 4.336475849151611
        entropy_coeff: 0.0
        kl: 0.009

#### 